In [86]:
import pandas as pd

df = pd.read_pickle('../data/final_data.pkl')

In [87]:
# # ElasticNet PREDICTION TOUS les Types => Pas très bon

# import numpy as np
# from sklearn.linear_model import ElasticNet
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import r2_score


# df_test = df.copy()

# X = df_test.drop(columns=['Valeur_fonciere','Commune'])
# print(X.columns)
# y = df_test['Valeur_fonciere']

# # Divisez les données en ensembles d'entraînement et de test
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Définissez les paramètres pour la recherche sur la grille
# param_grid = {
#     'alpha': [0.1, 1.0, 10.0],  # Valeurs à tester pour le paramètre alpha
#     'l1_ratio': [0.1, 0.5, 0.9]  # Valeurs à tester pour le paramètre l1_ratio
# }

# # Créez un modèle Elastic Net
# elastic_net = ElasticNet()

# # Créez un objet GridSearchCV pour la recherche sur la grille
# grid_search = GridSearchCV(estimator=elastic_net, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# # Exécutez la recherche sur la grille sur les données d'entraînement
# grid_search.fit(X_train, y_train)

# # Obtenez les meilleurs paramètres et le meilleur modèle
# meilleurs_parametres = grid_search.best_params_
# meilleur_modele = grid_search.best_estimator_

# # Utilisez le meilleur modèle pour faire des prédictions sur les données de test
# predictions = meilleur_modele.predict(X_test)

# # Calculez l'erreur quadratique moyenne des prédictions de Ridge
# mse = mean_squared_error(y_test, predictions,squared=False)

# print("Meilleurs paramètres pour Elastic Net: ", meilleurs_parametres)
# # Affichez l'erreur quadratique moyenne
# print("Erreur Quadratique Moyenne (MSE) : ", mse)

# # Créez un DataFrame avec les valeurs prédites et les valeurs réelles
# resultats = pd.DataFrame(
#     {'Valeur_reelle': y_test, 'Valeur_predite': predictions})

# # Affichez les valeurs prédites par rapport aux valeurs réelles
# print(resultats)
# # Calculez le R carré
# r2 = r2_score(y_test, predictions)
# # Affichez le R carré
# print("R carré :", r2)

In [88]:
import pandas as pd
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error
from math import sqrt


valeurs_code_type_local = [1, 2, 3, 4]
alphas = [10.0, 0.1, 1.0]  # Toutes les valeurs d'alpha que vous voulez tester
l1_ratios = [0.1, 0.5, 0.9]  # Toutes les valeurs de l1_ratio que vous voulez tester

# Dictionnaire pour stocker les performances par modèle et par valeur de 'Code_type_local', 'alpha' et 'l1_ratio'
performances = {}

# Dictionnaire pour stocker les meilleurs modèles par Code_type_local
meilleurs_modeles = {}

# Boucle sur les valeurs d'alpha et de l1_ratio pour Elastic Net
for alpha in alphas:
    for l1_ratio in l1_ratios:
        for code_type_local in valeurs_code_type_local:
            # Filtrer le DataFrame pour la valeur spécifique de 'Code_type_local'
            sous_ensemble_df = df[df['Code_type_local'] == code_type_local]

            # Divisez les données en caractéristiques (X) et la variable cible (y)
            X = sous_ensemble_df.drop(columns=['Code_type_local', 'Valeur_fonciere','Commune'])  # Remplacez 'variable_cible' par le nom réel de votre variable cible
            y = sous_ensemble_df['Valeur_fonciere']

            # Créez le modèle Elastic Net avec les alphas et l1_ratio spécifiques
            elastic_net = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
            elastic_net.fit(X, y)
            elastic_net_predictions = elastic_net.predict(X)
            elastic_net_rmse = sqrt(mean_squared_error(y, elastic_net_predictions))

        
           # Vérifiez si c'est le meilleur modèle pour cette combinaison de paramètres et de Code_type_local
            if code_type_local not in meilleurs_modeles or elastic_net_rmse < meilleurs_modeles[code_type_local]['RMSE']:
                meilleurs_modeles[code_type_local] = {
                    'Modèle': elastic_net,
                    'RMSE': elastic_net_rmse,
                    'Alpha': alpha,
                    'L1_ratio': l1_ratio
                }

             # Enregistrez les performances dans le dictionnaire
            performances[f'ElasticNet alpha={alpha} l1_ratio={l1_ratio}'][f'Code_type_local {code_type_local}'] = elastic_net_rmse
            performances[f'ElasticNet alpha={alpha} l1_ratio={l1_ratio}'] = {f'Code_type_local {code_type_local}': elastic_net_rmse}


# Boucle sur les valeurs d'alpha pour Lasso et Ridge
for alpha in alphas:
    for code_type_local in valeurs_code_type_local:
        # Filtrer le DataFrame pour la valeur spécifique de 'Code_type_local'
        sous_ensemble_df = df[df['Code_type_local'] == code_type_local]

        # Divisez les données en caractéristiques (X) et la variable cible (y)
        X = sous_ensemble_df.drop(columns=['Code_type_local', 'Valeur_fonciere','Commune'])  # Remplacez 'variable_cible' par le nom réel de votre variable cible
        y = sous_ensemble_df['Valeur_fonciere']

        # Créez le modèle Lasso avec l'alpha spécifique
        lasso = Lasso(alpha=alpha)
        lasso.fit(X, y)
        lasso_predictions = lasso.predict(X)
        lasso_rmse = sqrt(mean_squared_error(y, lasso_predictions))


        # Vérifiez si c'est le meilleur modèle pour cette combinaison de paramètres et de Code_type_local
        if code_type_local not in meilleurs_modeles or lasso_rmse < meilleurs_modeles[code_type_local]['RMSE']:
            meilleurs_modeles[code_type_local] = {
                'Modèle': lasso,
                'RMSE': lasso_rmse,
                'Alpha': alpha
            }

        # Créez le modèle Ridge avec l'alpha spécifique
        ridge = Ridge(alpha=alpha)
        ridge.fit(X, y)
        ridge_predictions = ridge.predict(X)
        ridge_rmse = sqrt(mean_squared_error(y, ridge_predictions))

       

        # Vérifiez si c'est le meilleur modèle pour cette combinaison de paramètres et de Code_type_local
        if code_type_local not in meilleurs_modeles or ridge_rmse < meilleurs_modeles[code_type_local]['RMSE']:
            meilleurs_modeles[code_type_local] = {
                'Modèle': ridge,
                'RMSE': ridge_rmse,
                'Alpha': alpha
            }

         # Enregistrez les performances dans le dictionnaire
        #performances[f'Lasso alpha={alpha}'][f'Code_type_local {code_type_local}'] = lasso_rmse
        performances[f'Lasso alpha={alpha}'] = {f'Code_type_local {code_type_local}': lasso_rmse}
        #performances[f'Ridge alpha={alpha}'][f'Code_type_local {code_type_local}'] = ridge_rmse
        performances[f'Ridge alpha={alpha}'] = {f'Code_type_local {code_type_local}': ridge_rmse}
       

# Créez un DataFrame à partir du dictionnaire de performances
df_performances = pd.DataFrame(performances)

# Résultat : meilleurs modèles par Code_type_local avec les paramètres optimaux et les modèles eux-mêmes
print(meilleurs_modeles)

{1: {'Modèle': Ridge(alpha=0.1), 'RMSE': 75362.45276465814, 'Alpha': 0.1}, 2: {'Modèle': Ridge(alpha=0.1), 'RMSE': 78334.70407894322, 'Alpha': 0.1}, 3: {'Modèle': Ridge(alpha=0.1), 'RMSE': 112029.67448943754, 'Alpha': 0.1}, 4: {'Modèle': Ridge(alpha=0.1), 'RMSE': 122762.97186939823, 'Alpha': 0.1}}


In [94]:
print(X.columns)

Index(['Surface_reelle_bati', 'Nombre_pieces_principales', 'Surface_terrain',
       'month', 'latitude', 'longitude', 'niveau_vie_commune',
       'Prix_moyen_m2'],
      dtype='object')


In [93]:
# Affichez le tableau récapitulatif des performances
df_performances.head()


,ElasticNet alpha=10.0 l1_ratio=0.1,ElasticNet alpha=10.0 l1_ratio=0.5,ElasticNet alpha=10.0 l1_ratio=0.9,ElasticNet alpha=0.1 l1_ratio=0.1,ElasticNet alpha=0.1 l1_ratio=0.5,ElasticNet alpha=0.1 l1_ratio=0.9,ElasticNet alpha=1.0 l1_ratio=0.1,ElasticNet alpha=1.0 l1_ratio=0.5,ElasticNet alpha=1.0 l1_ratio=0.9,Lasso alpha=10.0,Ridge alpha=10.0,Lasso alpha=0.1,Ridge alpha=0.1,Lasso alpha=1.0,Ridge alpha=1.0
Code_type_local 4,122850.567295,122845.685652,122829.559708,122777.591668,122769.771755,122763.428235,122827.814123,122816.677782,122779.48083,122762.97516,122762.971884,122762.97187,122762.971869,122762.971902,122762.97187


In [90]:
# Utilisez le modèle avec le RMSE minimum pour faire des prédictions
modele_code_type_1 = meilleurs_modeles[1]['Modèle']
modele_code_type_2 = meilleurs_modeles[2]['Modèle']
modele_code_type_3 = meilleurs_modeles[3]['Modèle']
modele_code_type_4 = meilleurs_modeles[4]['Modèle']

In [91]:
# Export Model

import pickle as pkl

with open('../model/modele_regression_code_1.pkl','wb') as fichier_pickler:
    pkl.dump(modele_code_type_1,fichier_pickler)

with open('../model/modele_regression_code_2.pkl','wb') as fichier_pickler:
    pkl.dump(modele_code_type_2,fichier_pickler)

with open('../model/modele_regression_code_3.pkl','wb') as fichier_pickler:
    pkl.dump(modele_code_type_3,fichier_pickler)

with open('../model/modele_regression_code_4.pkl','wb') as fichier_pickler:
    pkl.dump(modele_code_type_4,fichier_pickler)
